# Making the Gini Variable




with data from:

https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads

In [35]:
import pandas as pd
import numpy as np


In [5]:


url = 'http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2022.txt'

# Read in the data
data = pd.read_csv(url, sep=',', header=None, encoding='latin1')

# assign column names (based on official schema)
data.columns = [
    "Transaction unique identifier",
    "Price",
    "Date of Transfer",
    "Postcode",
    "Property Type",
    "Old/New",
    "Duration",
    "PAON",
    "SAON",
    "Street",
    "Locality",
    "Town/City",
    "District",
    "County",
    "PPD Category Type",
    "Record Status"
]

# Show first few rows
display(data)


,Transaction unique identifier,Price,Date of Transfer,Postcode,Property Type,Old/New,Duration,PAON,SAON,Street,Locality,Town/City,District,County,PPD Category Type,Record Status
0,{045A1898-4ABF-9A24-E063-4804A8C048EA},407400,2022-04-28 00:00,LU7 3FZ,S,Y,F,68,NaN,RAMSAY DRIVE,NaN,LEIGHTON BUZZARD,CENTRAL BEDFORDSHIRE,CENTRAL BEDFORDSHIRE,A,A
1,{045A1898-4AC1-9A24-E063-4804A8C048EA},357000,2022-05-27 00:00,LU7 3QS,S,Y,F,44,NaN,CHADWICK CRESCENT,NaN,LEIGHTON BUZZARD,CENTRAL BEDFORDSHIRE,CENTRAL BEDFORDSHIRE,A,A
2,{045A1898-4AC2-9A24-E063-4804A8C048EA},372950,2022-04-28 00:00,LU5 6TD,S,Y,F,11,NaN,SKYE GARDENS,HOUGHTON REGIS,DUNSTABLE,CENTRAL BEDFORDSHIRE,CENTRAL BEDFORDSHIRE,A,A
3,{045A1898-4AC4-9A24-E063-4804A8C048EA},570000,2022-05-31 00:00,SG18 9RF,D,Y,F,8,NaN,HARVEST MOUSE PLACE,LANGFORD,BIGGLESWADE,CENTRAL BEDFORDSHIRE,CENTRAL BEDFORDSHIRE,A,A
4,{045A1898-4AC5-9A24-E063-4804A8C048EA},570000,2022-05-27 00:00,LU7 3QS,D,Y,F,42,NaN,CHADWICK CRESCENT,NaN,LEIGHTON BUZZARD,CENTRAL BEDFORDSHIRE,CENTRAL BEDFORDSHIRE,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068640,{E2D14905-55E3-4C2D-E053-6B04A8C0422B},432000,2022-05-20 00:00,RM9 5UL,T,N,F,59,NaN,CONNOR ROAD,NaN,DAGENHAM,BARKING AND DAGENHAM,GREATER LONDON,A,A
1068641,{E2D14905-55E4-4C2D-E053-6B04A8C0422B},510000,2022-04-29 00:00,E4 8PH,T,N,F,27,NaN,SINCLAIR ROAD,NaN,LONDON,WALTHAM FOREST,GREATER LONDON,A,A
1068642,{E2D14905-55E5-4C2D-E053-6B04A8C0422B},635000,2022-05-23 00:00,E17 4PN,T,N,F,133,NaN,CHINGFORD ROAD,NaN,LONDON,WALTHAM FOREST,GREATER LONDON,A,A
1068643,{E2D14905-55E6-4C2D-E053-6B04A8C0422B},895000,2022-04-01 00:00,IG8 9EH,T,N,F,35,NaN,ST ALBANS CRESCENT,NaN,WOODFORD GREEN,REDBRIDGE,GREATER LONDON,A,A


## Filter London and Merseyside
smaller data will be easier to wrangle

In [24]:
# Filter the data for the specific counties
data = data[(data['County'] == 'GREATER LONDON') | (data['County'] == 'MERSEYSIDE')]

# Drop rows where 'Postcode' is NaN
data.dropna(subset=['Postcode'], inplace=True)

# Display the modified DataFrame
data = data[['Price', 'Postcode', 'County']]


data


,Price,Postcode,County
30,215000,SM6 8AR,GREATER LONDON
31,315000,CR4 1FD,GREATER LONDON
32,890000,BR4 9NW,GREATER LONDON
33,225000,CR2 6AE,GREATER LONDON
34,412500,BR1 1AG,GREATER LONDON
...,...,...,...
1068640,432000,RM9 5UL,GREATER LONDON
1068641,510000,E4 8PH,GREATER LONDON
1068642,635000,E17 4PN,GREATER LONDON
1068643,895000,IG8 9EH,GREATER LONDON


## Map to LSOA

https://borders.ukdataservice.ac.uk/

In [55]:
post_code_map = pd.read_csv('../../Data/LSOA_data/PostCodes/ONSPD_FEB_2022_UK.csv')
post_code_map

C:\Users\15kof\AppData\Local\Temp\ipykernel_22336\1738312960.py:1: DtypeWarning: Columns (18,31,40,45) have mixed types. Specify dtype option on import or set low_memory=False.
  post_code_map = pd.read_csv('../../Data/LSOA_data/PostCodes/ONSPD_FEB_2022_UK.csv')


,pcd,pcd2,pcds,dointr,doterm,oscty,ced,oslaua,osward,parish,...,ru11ind,oac11,lat,long,lep1,lep2,pfa,imd,calncv,stp
0,AB1 0AA,AB1 0AA,AB1 0AA,198001,199606.0,S99999999,S99999999,S12000033,S13002843,S99999999,...,3.0,1C3,57.101474,-2.242851,S99999999,S99999999,S23000009,6715,S99999999,S99999999
1,AB1 0AB,AB1 0AB,AB1 0AB,198001,199606.0,S99999999,S99999999,S12000033,S13002843,S99999999,...,3.0,1C3,57.102554,-2.246308,S99999999,S99999999,S23000009,6715,S99999999,S99999999
2,AB1 0AD,AB1 0AD,AB1 0AD,198001,199606.0,S99999999,S99999999,S12000033,S13002843,S99999999,...,3.0,6A1,57.100556,-2.248342,S99999999,S99999999,S23000009,6715,S99999999,S99999999
3,AB1 0AE,AB1 0AE,AB1 0AE,199402,199606.0,S99999999,S99999999,S12000034,S13002864,S99999999,...,6.0,1A2,57.084444,-2.255708,S99999999,S99999999,S23000009,5069,S99999999,S99999999
4,AB1 0AF,AB1 0AF,AB1 0AF,199012,199207.0,S99999999,S99999999,S12000033,S13002843,S99999999,...,3.0,6A4,57.096656,-2.258102,S99999999,S99999999,S23000009,6253,S99999999,S99999999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2673013,ZE3 9JW,ZE3 9JW,ZE3 9JW,198001,NaN,S99999999,S99999999,S12000027,S13002776,S99999999,...,8,1B1,59.873651,-1.305697,S99999999,S99999999,S23000009,4141,S99999999,S99999999
2673014,ZE3 9JX,ZE3 9JX,ZE3 9JX,198001,NaN,S99999999,S99999999,S12000027,S13002776,S99999999,...,8,1B1,59.875286,-1.307502,S99999999,S99999999,S23000009,4141,S99999999,S99999999
2673015,ZE3 9JY,ZE3 9JY,ZE3 9JY,198001,NaN,S99999999,S99999999,S12000027,S13002776,S99999999,...,8,1A1,59.891572,-1.313847,S99999999,S99999999,S23000009,4141,S99999999,S99999999
2673016,ZE3 9JZ,ZE3 9JZ,ZE3 9JZ,198001,NaN,S99999999,S99999999,S12000027,S13002776,S99999999,...,8,1A1,59.892392,-1.310899,S99999999,S99999999,S23000009,4141,S99999999,S99999999


In [57]:
post_code_map = post_code_map[['pcds', 'lsoa11']]

post_code_map

,pcds,lsoa11
0,AB1 0AA,S01006514
1,AB1 0AB,S01006514
2,AB1 0AD,S01006514
3,AB1 0AE,S01006853
4,AB1 0AF,S01006511
...,...,...
2673013,ZE3 9JW,S01012387
2673014,ZE3 9JX,S01012387
2673015,ZE3 9JY,S01012387
2673016,ZE3 9JZ,S01012387


In [59]:
gini_data = pd.merge(data, post_code_map, how='left', left_on='Postcode', right_on='pcds')

gini_data = gini_data[['lsoa11', 'Price', 'County']]

gini_data.columns = ['LSOA', 'Price', 'County']

gini_data

,LSOA,Price,County
0,E01004172,215000,GREATER LONDON
1,E01003435,315000,GREATER LONDON
2,E01000840,890000,GREATER LONDON
3,E01001040,225000,GREATER LONDON
4,E01000675,412500,GREATER LONDON
...,...,...,...
146383,E01000065,432000,GREATER LONDON
146384,E01004454,510000,GREATER LONDON
146385,E01004460,635000,GREATER LONDON
146386,E01003670,895000,GREATER LONDON


In [60]:
def gini(x):
    x = np.sort(x)
    n = len(x)
    index = np.arange(1, n+1)
    return (2 * np.sum(index * x)) / (n * np.sum(x)) - (n + 1) / n

gini = (
    gini_data
    .groupby(['County', 'LSOA'])['Price']
    .apply(gini) 
    .reset_index(name='gini')
)


In [ ]:
gini

,County,LSOA,gini
0,GREATER LONDON,E01000001,0.216543
1,GREATER LONDON,E01000002,0.224353
2,GREATER LONDON,E01000003,0.170370
3,GREATER LONDON,E01000005,0.631699
4,GREATER LONDON,E01000006,0.261445
...,...,...,...
5744,MERSEYSIDE,E01033764,0.287901
5745,MERSEYSIDE,E01033765,0.288609
5746,MERSEYSIDE,E01033766,0.099511
5747,MERSEYSIDE,E01033767,0.101026


In [62]:
gini.to_csv('../../Data/Data Sets/gini_dataset.csv', index=False)